In [11]:
import pandas as pd

df = pd.read_csv('cleaned.csv')

df

,Upvotes,Date,Title,Body Text,Link
0,10,1449121338,Breaking Through the Safeties,Finally heard back from all of my safeties! O...,https://www.reddit.com/r/collegeresults/commen...
1,9,1449874637,Virginia Tech ED Deferred :(,Demographics: In-state Asian male\n\nIntended ...,https://www.reddit.com/r/collegeresults/commen...
2,6,1449888267,Accepted To Middlebury ED,Middlebury ED (only application)\n\nno financi...,https://www.reddit.com/r/collegeresults/commen...
3,6,1449900824,Accepted to Vanderbilt School of Engineering ED,Schools: Vanderbilt University (Accepted ED)\n...,https://www.reddit.com/r/collegeresults/commen...
4,7,1449936672,Accepted into Duke Pratt ED,I was just accepted this past Thursday to my f...,https://www.reddit.com/r/collegeresults/commen...
...,...,...,...,...,...
5598,1,1720456114,This Sub as issues...,Why is it that a black//Latine person will sha...,https://www.reddit.com/r/collegeresults/commen...
5599,1,1720471080,EU student bags ivies and top schools,**Demographics**\n\n* Gender: Male \n* Race/Et...,https://www.reddit.com/r/collegeresults/commen...
5600,1,1720484984,"Great Supplementals, Got into T20s",**Demographics**\n\n* Gender: Female\n* Race/E...,https://www.reddit.com/r/collegeresults/commen...
5601,1,1720485521,"Psych Major, Got into T20s, Great Essays",**Demographics**\n\n* Gender: Female\n* Race/E...,https://www.reddit.com/r/collegeresults/commen...


In [12]:
df2 = df
df2.drop(['Upvotes', 'Date', 'Title'], axis=1, inplace=True)


#df2 = df2.tail(10) #only lsat 10 posts

df2


,Body Text,Link
0,Finally heard back from all of my safeties! O...,https://www.reddit.com/r/collegeresults/commen...
1,Demographics: In-state Asian male\n\nIntended ...,https://www.reddit.com/r/collegeresults/commen...
2,Middlebury ED (only application)\n\nno financi...,https://www.reddit.com/r/collegeresults/commen...
3,Schools: Vanderbilt University (Accepted ED)\n...,https://www.reddit.com/r/collegeresults/commen...
4,I was just accepted this past Thursday to my f...,https://www.reddit.com/r/collegeresults/commen...
...,...,...
5598,Why is it that a black//Latine person will sha...,https://www.reddit.com/r/collegeresults/commen...
5599,**Demographics**\n\n* Gender: Male \n* Race/Et...,https://www.reddit.com/r/collegeresults/commen...
5600,**Demographics**\n\n* Gender: Female\n* Race/E...,https://www.reddit.com/r/collegeresults/commen...
5601,**Demographics**\n\n* Gender: Female\n* Race/E...,https://www.reddit.com/r/collegeresults/commen...


In [13]:
import re
import pandas as pd

def extractData(row, pattern, matchGroup=1):
    text = str(row['Body Text'])
    pattern = re.compile(pattern, re.IGNORECASE)
    match = pattern.search(text)
    
    if match:
        return match.group(matchGroup)  # Don't include the search part before it, ONLY the captured GROUP!
    else:
        return -1


def extractWithIndex(row, start, end):
    
    #prep
    text = str(row['Body Text'])
    text = text.lower()
    start = start.lower()

    start = text.find(start)


    #end is an array of potential ends. we want the first index that is not -1
    actualMinEnd = -1
    for i in end:
        
        if i == "END":
            #instead of finding the next occurence, we just want to find the end of the text
            actualMinEnd = len(text)
            break

        i = i.lower()
        end = text.find(i)
        if end != -1:
            actualMinEnd = end
            break
    
            
    if start != -1 and actualMinEnd != -1:
        return text[start:actualMinEnd]
    else:
        return -1
    


new_df = pd.DataFrame()

new_df['Body Text'] = df2['Body Text']
new_df['Link'] = df2['Link']

#DEMOGRAPHICS
new_df['Gender'] = df2.apply(lambda row: extractData(row, r"Gender:\s*(.*)"), axis=1)
new_df['Race'] = df2.apply(lambda row: extractData(row, r"Race\/Ethnicity:\s*(.*)"), axis=1)
new_df['Residence'] = df2.apply(lambda row: extractData(row, r"Residence:\s*(.*)"), axis=1)
new_df['Income'] = df2.apply(lambda row: extractData(row, r"Income Bracket:\s*(.*)"), axis=1)
new_df['School Type'] = df2.apply(lambda row: extractData(row, r"Type of School:\s*(.*)"), axis=1)
new_df['Hooks'] = df2.apply(lambda row: extractData(row, r"(?:Hooks \(Recruited Athlete, URM, First-Gen, Geographic, Legacy, etc.\)|Hooks):\s*(.*)"), axis=1)

#ACADEMICS
new_df['Major'] = df2.apply(lambda row: extractData(row, r"Intended Major\(s\)\**:*(.*)"), axis=1)
new_df['GPA'] = df2.apply(lambda row: extractData(row, r"GPA \(UW\/W\)\**:*(.*)"), axis=1)
new_df['Rank'] = df2.apply(lambda row: extractData(row, r"Rank \(or percentile\):(.*)"), axis=1)
new_df['APs'] = df2.apply(lambda row: extractData(row, r".*Enrollment\/etc:(.*)"), axis=1)
new_df['SeniorCourseLoad'] = df2.apply(lambda row: extractData(row, r".*Course Load:(.*)"), axis=1)


#testing
new_df['SAT'] = df2.apply(lambda row: extractData(row, r"SAT.*:(.*)"), axis=1)
new_df['ACT'] = df2.apply(lambda row: extractData(row, r"ACT.*:(.*)"), axis=1)

#activities
new_df['EC'] = df2.apply(lambda row: extractWithIndex(row, "Extracurriculars/Activities", ["Awards/Honor", "Letters of Recommendation", "Interviews", "Essays", "Decisions", "Acceptances" ]), axis=1)
new_df['Awards'] = df2.apply(lambda row: extractWithIndex(row, "Awards/Honor", ["Letters of Recommendation", "Interviews", "Essays", "Decisions", "Acceptances"] ), axis=1)
new_df['LOR'] = df2.apply(lambda row: extractWithIndex(row, "Letters of Recommendation", [ "Interviews", "Essays","Decisions", "Acceptances"]), axis=1)
new_df['Interviews'] = df2.apply(lambda row: extractWithIndex(row, "Interviews", ["Essays", "Decisions", "Acceptances"]), axis=1)
new_df['Essays'] = df2.apply(lambda row: extractWithIndex(row, "Essays", ["Decisions", "Acceptances"]), axis=1)


#decisions
new_df['Acceptances'] = df2.apply(lambda row: extractWithIndex(row, "Acceptances", ["Waitlists", "Rejections", "Additional Information"]), axis=1)
new_df['Waitlists'] = df2.apply(lambda row: extractWithIndex(row, "Waitlists", ["Rejections", "Additional Information"]), axis=1)
new_df['Rejections'] = df2.apply(lambda row: extractWithIndex(row, "Rejections", ["Additional Information", "END"]), axis=1)


new_df
new_df.to_csv('extracted.csv', index=False)


